This code iteratively solves for the correct ride and roll rate using the method described in section 16 of RCVD

In [10]:
import numpy as np
import pandas as pd
import pickle

# Importing Engine Data from Excel using PANDAS
cbf650r_engine_data_file_path = "C:/Users/maxwe/Downloads/FSAE/2023-2024 Car/Repo/engine_data.xlsx"
cbf650r_engine_data = pd.read_excel(cbf650r_engine_data_file_path)

# Importing Car Model from car_model using PICKLE
with open('C:/Users/maxwe/Downloads/FSAE/2023-2024 Car/Repo/car_model.pkl', 'rb') as f:
    car_var = pickle.load(f)

# total weight of car (minus driver) (lbm)
W_T = car_var['W_T']
# weight bias, if less than 0.5, then the rear of the car will have more weight, if more than 0.5, then the front will have more weight
W_bias = car_var['W_bias']
# length of wheelbase (in)
l = car_var['l']
# vertical center of gravity (in)
h = car_var['h']
# setting pi as a number
pi = 3.14159
# tire grip limit (G's)
A_y = car_var['tire_a']
# front track width (in)
t_f = car_var['t_f']
# rear track width (in)
t_r = car_var['t_r']
# CG to roll axis (in)
H = car_var['H']
# distance between CG and front track (in)
a = car_var['a']
# distance between CG and rear track (in)
b = car_var['b']
# front roll axis height (in)
z_rf = car_var['z_rf']
# rear roll axis height (in)
z_rr = car_var['z_rr']
# Tire rates, camper from 0-4 deg, (lb/in)
T_r = car_var['T_r']
T_r0 = T_r[0]
T_r1 = T_r[1]
T_r2 = T_r[2]
T_r3 = T_r[3]
T_r4 = T_r[4]


W_f = W_bias*W_T # lbs, weight over front wheels
W_r = (1-W_bias)*W_T# lbs, weight over rear wheels
W_1 = W_f/2 #lbs, front left
W_2 = W_f/2 #lbs, front right
W_3 = W_r/2 #lbs, rear left
W_4 = W_r/2 #lbs, rear right
MR_F = 1.1786 # Motion Ratio for Front
MR_R = 1.1786 # Motion Ratio for Rear


K_RF = 167 # lb/in, initial ride rates so that the code can run
K_RR = 162

y_F_ideal = 0.65 # This is the ideal wheel travel at maximum lateral acceleration 
y_R_ideal = 0.65

y_F = 1 # We start with a wheel travel displacement above ideal so the while loop runs properly
y_R = 1

while y_R > y_R_ideal:

    K_rollF = 12*K_RF*t_f**2/2 # lb*ft/rad, roll rate
    K_rollR = 12*K_RR*t_r**2/2

    freq_f = 1/(2*np.pi)*np.sqrt(K_RF*12*32.2/W_2) # natrual frequency of ride rate
    freq_r = 1/(2*np.pi)*np.sqrt(K_RR*12*32.2/W_4)

    W_F = A_y*W_T/t_f*(H*K_rollF/(K_rollF+K_rollR)+b/l*z_rf) # weight transfer for front and rear tracks
    W_R = A_y*W_T/t_r*(H*K_rollR/(K_rollF+K_rollR)+a/l*z_rr)

    y_F = W_F/K_RF # wheel travel at maximum lateral acceleration
    y_R = W_R/K_RR

    K_RF = W_F/y_F_ideal # New ride rate value
    K_RR = W_R/y_R_ideal

# Calculating Wheel Rate
K_WF = T_r1*K_RF/(T_r1 - K_RF) # Calculated at a camber angle of 1 deg
K_WR = T_r1*K_RR/(T_r1 - K_RR)

# Calculating Spring Rate -- The Numerator is equivalent to the force the spring will see when the total axle displacement
#                            in the z-direction is 1 in. The denominator is the distance that the spring will compress when 
#                            the total axle displacement is 1in in the z-direction.
K_SF = K_WF*(1.9182/3.2622)/(1/MR_F)
K_SR = K_WR*(1.9182/3.2622)/(1/MR_R)

# Printing Values in easily readable form
print(f"The front ideal ride rate is {K_RF:.5} lb/in, the rear ride rate is {K_RR:.5} lb/in\n")
print(f"The front ideal wheel rate is {K_WF:.5} lb/in, the rear wheel rate is {K_WR:.5} lb/in\n")
print(f"The front ideal spring rate is {K_SF:.5} lb/in, the rear spring rate is {K_SR:.5} lb/in\n")
print(f"the front ideal wheel travel at {A_y:.5} G's in the z-axis is {y_F:.5} in, and the rear wheel travel is {y_R:.5} in\n")
print(f"The front ideal natural frequency is {freq_f:.5} Hz, the rear natural frequency is {freq_r:.5} Hz\n")
print(f"The front ideal roll rate is {K_rollF:.5} lb*ft/rad, and the rear roll rate is {K_rollR:.5} lb*ft/rad\n")
print(f"The front ideal roll rate is {K_rollF*pi/180:.5} lb*ft/deg, and the rear roll rate is {K_rollR*pi/180:.5} lb*ft/deg\n")
print(f"The front ideal weight transfer is {W_F:.5} lb, and the rear weight transfer is {W_R:.5} lb\n\n")

# Setting the spring rates to realistic values to determine what the actual ride and roll rates will be
K_SF_real = 200 # lb/in
K_SR_real = 300 # lb/in

# Calculating actual wheel rates
K_WF_real = K_SF_real*(1/MR_F)/(1.9182/3.2622)
K_WR_real = K_SR_real*(1/MR_R)/(1.9182/3.2622)

# Calculating actual ride rates
K_RF_real = K_WF_real*T_r1/(K_WF_real+T_r1)
K_RR_real = K_WR_real*T_r1/(K_WR_real+T_r1)

# Calculating actual roll rates
K_rollF_real = 12*K_RF_real*t_f**2/2
K_rollR_real = 12*K_RR_real*t_r**2/2

# Calculating actual wheel travel
W_F_real = A_y*W_T/t_f*(H*K_rollF_real/(K_rollF_real+K_rollR_real)+b/l*z_rf) # weight transfer for front and rear tracks
W_R_real = A_y*W_T/t_r*(H*K_rollR_real/(K_rollF_real+K_rollR_real)+a/l*z_rr)

# Calculating actual wheel displacement
y_F_real = W_F_real/K_RF_real 
y_R_real = W_R_real/K_RR_real

# Calculating actual natural frequencies
freq_f_real = 1/(2*np.pi)*np.sqrt(K_RF_real*12*32.2/W_2) # natrual frequency of ride rate
freq_r_real = 1/(2*np.pi)*np.sqrt(K_RR_real*12*32.2/W_4)

# Printing Values in easily readable form
print(f"The front actual ride rate is {K_RF_real:.5} lb/in, the rear ride rate is {K_RR_real:.5} lb/in\n")
print(f"The front actual wheel rate is {K_WF_real:.5} lb/in, the rear wheel rate is {K_WR_real:.5} lb/in\n")
print(f"The front actual spring rate is {K_SF_real} lb/in, the rear spring rate is {K_SR_real} lb/in\n")
print(f"the actual front wheel travel at {A_y:.5} G's in the z-axis is {y_F_real:.5} in, and the rear wheel travel is {y_R_real:.5} in\n")
print(f"The actual front natural frequency is {freq_f_real:.5} Hz, the rear natural frequency is {freq_r_real:.5} Hz\n")
print(f"The actual front roll rate is {K_rollF_real:.5} lb*ft/rad, and the rear roll rate is {K_rollR_real:.5} lb*ft/rad\n")
print(f"The actual front roll rate is {K_rollF_real*pi/180:.5} lb*ft/deg, and the rear roll rate is {K_rollR_real*pi/180:.5} lb*ft/deg\n")
print(f"The actual front weight transfer is {W_F_real:.5} lb, and the rear weight transfer is {W_R_real:.5} lb\n\n")







The front ideal ride rate is 212.15 lb/in, the rear ride rate is 280.22 lb/in

The front ideal wheel rate is 297.91 lb/in, the rear wheel rate is 452.11 lb/in

The front ideal spring rate is 206.46 lb/in, the rear spring rate is 313.33 lb/in

the front ideal wheel travel at 1.4 G's in the z-axis is 0.65 in, and the rear wheel travel is 0.65 in

The front ideal natural frequency is 3.307 Hz, the rear natural frequency is 3.8542 Hz

The front ideal roll rate is 3.1607e+06 lb*ft/rad, and the rear roll rate is 3.9761e+06 lb*ft/rad

The front ideal roll rate is 5.5164e+04 lb*ft/deg, and the rear roll rate is 6.9395e+04 lb*ft/deg

The front ideal weight transfer is 137.9 lb, and the rear weight transfer is 182.14 lb


The front actual ride rate is 207.38 lb/in, the rear ride rate is 272.71 lb/in

The front actual wheel rate is 288.59 lb/in, the rear wheel rate is 432.88 lb/in

The front actual spring rate is 200 lb/in, the rear spring rate is 300 lb/in

the actual front wheel travel at 1.4 G